In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('HomeBridger Data Feb112025 (Do not edit).xlsx', sheet_name=None)

In [ ]:

client_goal_df = df['CLIENT_GOAL__C']
program_engagement_df = df["PMDM__PROGRAMENGAGEMENT__C"]
program_engagement_df = program_engagement_df.drop_duplicates(subset=['EXTERNAL_ID__C'], keep='first') ## get rid of external id duplicates (since only diffferece in RECORD_SOURCE__C)
program_engagement_df = program_engagement_df[program_engagement_df['EX_FAMILY_COLLECTIVE_ENROLLMENT__C'].isna()] ## only keep the head of the family or individual (not the family member) since only these have linked client goals

merged_df = client_goal_df.merge(program_engagement_df,
                                 left_on="EXT_HOUSEHOLD__C",
                                 right_on="EXTERNAL_ID__C", #EX_FAMILY_COLLECTIVE_ENROLLMENT__C
                                 how="inner") # since we want to keep only the rows that are in both dataframes


merged_df = client_goal_df.merge(program_engagement_df,
                                 left_on="EXT_HOUSEHOLD__C",
                                 right_on="EXTERNAL_ID__C", #EX_FAMILY_COLLECTIVE_ENROLLMENT__C
                                 how="inner") # since we want to keep only the rows that are in both dataframes
merged_df = merged_df.dropna(axis=1, how='all')
 
filtered_df = merged_df[merged_df["GOAL_STATUS__C"].isin(["Achieved", "Not Achieved"])]
filtered_df["GOAL_STATUS_BINARY"] = filtered_df["GOAL_STATUS__C"].map({"Achieved": 1, "Not Achieved": 0})


drop_columns =  ['SYSTEMMODSTAMP_x', 'GOAL_STATUS__C', 'EXT_HOUSEHOLD__C', 'EXT_CLIENT__C', 'EXTERNAL_ID__C_x', 'RECORD_HASH__C_x', 'TFC_SYNC__C_x', 'EX_ACCOUNT__C', 'SYSTEMMODSTAMP_y', 'EX_CONTACT__C', 'PMDM__APPLICATIONDATE__C', 'LAST_NAME__C', 'VERBAL_CONSENT_GIVEN_SALESFORCE_ROI__C', 'FIRST_NAME__C', 'VERBAL_CONSENT_GIVEN_HMIS_ROI__C', 'EMAIL__C', 'BIRTHDATE__C', 'VERBAL_CONSENT_GIVEN_MNPS_ROI__C', 'HMIS_ROI__C', 'HMIS_SALESFORCE__C', 'MNPS_ROI__C', 'GENDER__C', 'RACE__C', 'ETHNICITY__C', 'VETERAN_STATUS__C', 'PHONE__C', 'CLIENT_DOESN_T_KNOW__C', 'RECORD_TYPE_NAME__C', 'ENROLLED_IN_TFC_CHILDCARE__C', 'ZIPCODE_AT_EXIT__C', 'ZIPCODE_AT_ENTRY__C', 'EX_PARTNER_AGENCY__C', 'CLOSURE_DATE__C', 'CLOSURE_REASON__C', 'EXTERNAL_ID__C_y', 'CLONED_FROM_ID__C', 'RECORD_HASH__C_y', 'TFC_SYNC__C_y', 'RECORD_SOURCE__C','PMDM__ENDDATE__C','RECORD_ORIGIN__C_x','LATEST_DATE_OF_HOMELESSNESS__C','PMDM__STARTDATE__C','DATE_GOAL_ACHIEVED__C']

categorical_columns = [
   'DOMAIN__C', 'OUTCOME__C',
   'SPECIFIC_GOAL_ACTION_STEP__C', 'PMDM__STAGE__C',
   'REASON_CLOSED__C', 'LIVING_SITUATION_AT_EXIT__C',
   'LIVING_SITUATION_PRIOR_TO_ENTRY__C', 'PREGNANCY_STATUS_AT_ENROLLMENT__C',
   'LIVING_SITUATION_AT_ENTRY__C', 'SUBSIDY_AT_EXIT__C', 'COUNTY_AT_ENTRY__C',
   'COUNTY_AT_EXIT__C', 'RECORD_ORIGIN__C_y', 'TIME_TO_COMPLETE'
]

dropped_df = filtered_df.drop(columns=drop_columns)
dropped_df['DATE_GOAL_SET__C'] = pd.to_datetime(dropped_df['DATE_GOAL_SET__C'], errors='coerce')
dropped_df['TARGET_COMPLETION_DATE__C'] = pd.to_datetime(dropped_df['TARGET_COMPLETION_DATE__C'], errors='coerce')
# compute the time difference between the target completion date and the date the goal was set
dropped_df['TIME_TO_COMPLETE'] = (dropped_df['TARGET_COMPLETION_DATE__C'] - dropped_df['DATE_GOAL_SET__C']).dt.days
dropped_df['TIME_TO_COMPLETE'] = dropped_df['TIME_TO_COMPLETE'].fillna(0)
dropped_df['TIME_TO_COMPLETE'] = dropped_df['TIME_TO_COMPLETE'].astype(int)
dropped_df.drop(columns=['DATE_GOAL_SET__C', 'TARGET_COMPLETION_DATE__C'], inplace=True)
dropped_df.fillna('Missing', inplace=True)

In [ ]:
dropped_df['RECORD_ORIGIN__C_y'].value_counts()

# MOBC, SL, UWGN

MOBC_df = dropped_df[dropped_df['RECORD_ORIGIN__C_y'] == 'MOBC']
CC_df = dropped_df[dropped_df['RECORD_ORIGIN__C_y'] == 'CC']
SL_df = dropped_df[dropped_df['RECORD_ORIGIN__C_y'] == 'SL']
UWGN_df = dropped_df[dropped_df['RECORD_ORIGIN__C_y'] == 'UWGN']
WE_df = dropped_df[dropped_df['RECORD_ORIGIN__C_y'] == 'WE']
MOBC_df.info()
MOBC_df.to_csv('MOBC.csv', index=False)
SL_df.to_csv('SL.csv', index=False)
UWGN_df.to_csv('UWGN.csv', index=False)
CC_df.to_csv('CC.csv', index=False)
WE_df.to_csv('WE.csv', index=False)
dropped_df.to_csv('all.csv', index=False)